In [3]:
# Set up modules
import pandas as pd
import sys
import requests
from xlsxwriter.utility import xl_rowcol_to_cell
sys.path.append(r'/home/jupyter/reusable_code')
import google_api_functions as gaf
import trello_generic as tg
import sqlite3
from google.cloud import bigquery # To run BQ statements
import re
# Set up SQL DB
conn = sqlite3.connect('SQL_connection1.db') #Create a connection object

# Set up credentials for Trello and Google 

# Google Sheets Credentials
creds=gaf.Authenticate_Google(r'/home/jupyter/reusable_code/') # Return logged-in credentials

# General setup and credentials: Trello
from trello import TrelloClient
trelloUserCreds=tg.readTrelloCredsFromFile(r'/home/jupyter/reusable_code/trellocreds.pickle')
mykey,mysecret,mytoken=trelloUserCreds

client = TrelloClient(api_key=mykey,api_secret=mysecret,token=mytoken)

# Return Trello board, client and other credentials objects. "myboard_creds" is a tuple of items which can be unpacked
# to cover off all of the various levels you might need access at
dataBoard,dataBoard_id,dataBoard_creds=tg.Return_board_by_name(mykey,mysecret,mytoken,"Data 2021")
researchBoard,researchBoard_id,researchBoard_creds=tg.Return_board_by_name(mykey,mysecret,mytoken,"Research 2021")
#oldBoard,oldBoard_id,oldBoard_creds=tg.Return_board_by_name(mykey,mysecret,mytoken,"Insights & Data")
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 500)

In [4]:
# Unpack board and credentials attributes
myboard, mykey,mysecret,mytoken,myboard_id, myboard_name,client=dataBoard_creds



my_cards=myboard.get_cards()
my_lists=myboard.list_lists()

In [42]:
lists_to_exclude=

In [47]:
type(my_cards)

In [46]:
len(lists_to_exclude)

In [54]:
acard=my_cards[55]

In [63]:
acard.name

In [55]:
acard.list_movements()

In [56]:
acard.listCardMove_date()

In [62]:
acard.latestCardMove_date

In [127]:
from datetime import datetime
import datetime as dt
import numpy as np
movements=acard.list_movements()
creation=acard.card_created_date

movements2=[{'from':i['source']['name'],\
            'to':i['destination']['name'],\
            'when':i['datetime'].replace(tzinfo=None)} for i in movements]

In [178]:
acard.get_list()

In [128]:
if len(movements2)==0:
    movements2=[{'from':'Nothing','to':acard.get_list().name,'when':acard.card_created_date},\
               {'from':acard.get_list().name,'to':'Refresh','when':datetime.now()}\
               ]

else:
    movements2.append({'from':'Nothing','to':movements2[-1]['from'], 'when':acard.card_created_date})
    movements2.append({'from':acard.get_list().name,'to':'Refresh','when':datetime.now()})
    

In [129]:
movement_df=pd.DataFrame(movements2).sort_values(by='when')

In [134]:
movement_df=movement_df.rename(columns={'when':'exitedTime','from':'enteredList','to':'exitedTo'})
movement_df['enteredTime']=movement_df['exitedTime'].shift(1)
movement_df['timeSpent']=(movement_df['exitedTime']-movement_df['enteredTime']).dt.days
movement_df['isCurrent']=movement_df['enteredList'].apply(lambda x: 1 if x==acard.get_list().name else 0)
movement_df

In [190]:
summary_df=movement_df[['isCurrent','enteredList','timeSpent','enteredTime','exitedTime']].groupby(['isCurrent','enteredList']).agg({'timeSpent' : [np.sum, 'count'],\
    'enteredTime' : [np.min, np.max],'exitedTime':'max'}).rename(columns={'sum':'timeSpent','count':'timesEntered','amin':'FirstEntered','amax':'LastEntered','max':'LastExited'}).droplevel(level=0,axis=1)
summary_df

In [ ]:

summary_dict=summary_df.reset_index().to_dict(orient='rows')
current_list_summary=[i for i in summary_dict if i['isCurrent']==1][0] 
current_list_summary

In [185]:
movement_df.reset_index(drop=True).to_dict(orient='rows')

In [59]:
help(acard)

In [44]:

my_cards=myboard.get_cards()
my_lists=myboard.list_lists()
list_names={i.id:i.name for i in my_lists} # List Name lookup
list_pos={i.id:n+1 for n,i in enumerate(my_lists)} # List Position lookup
cardpos=pd.DataFrame([{'cardid':card.id,'name':card.name, 'pos': card.pos, 'list':list_names[card.list_id],'listpos':list_pos[card.list_id]} for card in my_cards]).sort_values(by=['listpos','pos'])
cardpos['boardpos']=cardpos.reset_index().index+1
cardpos['cardpos']=cardpos.groupby('listpos').cumcount()+1
#cardpos=[i['listpos']:list_pos[i['list']] for i in cardpos ]
poslist=cardpos.drop(columns=['pos']).to_dict(orient='rows')